In [1]:
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import collections
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')
def RandomForestLDA(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ArousalEncoded'+".csv")
    ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceEncoded'+".csv")
    ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test , z_train,z_test= train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)
# Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
# LDA Classifier Valence=y
    lda = LDA(n_components=20)  
    X_train = lda.fit_transform(X_train, y_train)  
    X_test = lda.transform(X_test)  
    classifier = RandomForestClassifier(max_depth=2, random_state=0)
    classifier.fit(X_train, y_train)  
    y_predict = classifier.predict(X_test)
# LDA Classifier Arousal=z
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    lda = LDA(n_components=20)  
    X_train = lda.fit_transform(X_train, z_train)  
    X_test = lda.transform(X_test)  
    classifier = RandomForestClassifier(max_depth=2, random_state=0)
    classifier.fit(X_train, z_train)  
    z_predict = classifier.predict(X_test)
    cm = confusion_matrix(z_test, z_predict)
    #return f1 score and accurecies for valence and arousal
    return [accuracy_score(z_test, z_predict)*100,accuracy_score(y_test, y_predict)*100
            ,f1_score(z_test, z_predict, average='weighted'),
            f1_score(y_test, y_predict, average='weighted')]

############### Happy And Sad ####################
def RandomForestHS(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'HappyAndSadEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'HS'+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    while ((np.sum(y_train == 0) == 0) or (np.sum(y_train == 1) == 0)): #Don't Start training unless you have 2 classes.. happens with imbalance
        print("here")
        X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
# Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
# LDA Classifier Valence=y
    lda = LDA(n_components=20)  
    X_train = lda.fit_transform(X_train, y_train)  
    X_test = lda.transform(X_test)  
    classifier = RandomForestClassifier(max_depth=2, random_state=0)
    classifier.fit(X_train, y_train)  
    y_predict = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
#     print(cm)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]

############### All Emotions ####################
def RandomForestCombined(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceAndArousalEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
# Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
# LDA Classifier Valence=y
    lda = LDA(n_components=15)  
    X_train = lda.fit_transform(X_train, y_train)  
    X_test = lda.transform(X_test)  
    classifier = RandomForestClassifier(max_depth=10, random_state=0)
    classifier.fit(X_train, y_train)  
    y_predict = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
#     print(cm)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]
    
def Run100(UserName,Window=256,StartTime=0,EndTime=63):
    arousal=[]
    valence=[]
    F1_ScoreArousal=[]
    F1_ScoreValence=[]
    for i in range (100):
        RandomForest=RandomForestLDA(UserName,Window,StartTime,EndTime)
        arousal+=[RandomForest[0]]
        valence+=[RandomForest[1]]
        F1_ScoreArousal+=[RandomForest[2]]
        F1_ScoreValence+=[RandomForest[3]]
    print("Arousal "+str(np.mean(arousal)))
    print(collections.Counter(arousal))
    print("Valence "+str(np.mean(valence)))
    print(collections.Counter(valence))
    return [np.mean(arousal),np.mean(valence),np.mean(F1_ScoreArousal),np.mean(F1_ScoreValence)];
    

    
def RunCombined100(UserName,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_ALL=[]
    for i in range (100):
        AllEmotions=RandomForestCombined(UserName,Window,StartTime,EndTime)
        Combined+=[AllEmotions[0]]
        F1_ALL+=[AllEmotions[1]]
    print("Array of average All Emotions accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_ALL)];
    
def RunHS100(UserName,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_HS=[]
    for i in range (100):
        HS=RandomForestHS(UserName,Window,StartTime,EndTime)
        Combined+=[HS[0]]
        F1_HS+=[HS[1]]
    print("Array of average Happy Sad accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_HS)];

if __name__ == "__main__":
    HSAccurecy=0;
    HSF1=0;
    CombinedAccurecy=0;
    CombinedF1=0;
    ValenceAccurecy=0;
    ValenceF1=0;
    ArousalAccurecy=0;
    ArousalF1=0;
    for i in range(1,10):
        print("s0"+str(i))
        #Happy and Sad
        var =RunHS100("s0"+str(i))
        print("Happy Sad")
        print(var)
        HSAccurecy+=var[0]
        HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s0"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s0"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
        ValenceF1+=var2[3]
    for i in range(10,33):
        print("s"+str(i))
        #Happy and Sad
        if not ("s"+str(i) == "s31"):
            var =RunHS100("s"+str(i))
            print("Happy Sad")
            print(var)
            HSAccurecy+=var[0]
            HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
    print("Accurecy happy sad for 31 users "+str(HSAccurecy/31))
    print("F1 happy sad for 31 users "+str(HSF1/31))
    print("Accurecy Combined sad for 32 users"+str(CombinedAccurecy/32))
    print("F1 Combined sad for  32 users "+str(CombinedF1/32))
    print("Accurecy Valence for  32 users "+str(ValenceAccurecy/32))
    print("F1 Valence for 32 users "+str(ValenceF1/32))
    print("Accurecy Arousal for  32 users "+str(ArousalAccurecy/32))
    print("F1 Arousal for  32 users "+str(ArousalF1/32))



        

s01
Array of average Happy Sad accurecies for user s01
Counter({60.0: 37, 40.0: 29, 80.0: 22, 20.0: 7, 100.0: 4, 0.0: 1})
Happy Sad
[56.8, 0.5546349206349206]
Array of average All Emotions accurecies for user s01
Counter({25.0: 39, 37.5: 22, 50.0: 16, 12.5: 15, 0.0: 3, 62.5: 3, 75.0: 2})
Combined Emotions
[31.25, 0.3016815476190476]
Arousal 54.75
Counter({50.0: 28, 62.5: 22, 37.5: 18, 75.0: 12, 87.5: 9, 25.0: 9, 100.0: 1, 12.5: 1})
Valence 47.0
Counter({37.5: 26, 62.5: 23, 50.0: 23, 25.0: 12, 75.0: 9, 12.5: 6, 87.5: 1})
Arousal Valence
[54.75, 47.0, 0.5181890054390054, 0.46251262626262624]
s02
Array of average Happy Sad accurecies for user s02
Counter({80.0: 34, 60.0: 34, 40.0: 14, 100.0: 12, 20.0: 5, 0.0: 1})
Happy Sad
[66.2, 0.6520238095238095]
Array of average All Emotions accurecies for user s02
Counter({50.0: 28, 37.5: 24, 25.0: 22, 12.5: 11, 62.5: 10, 75.0: 2, 87.5: 2, 0.0: 1})
Combined Emotions
[39.375, 0.38443281024531023]
Arousal 58.125
Counter({50.0: 25, 62.5: 23, 75.0: 22, 3

Arousal 59.0
Counter({62.5: 40, 50.0: 20, 75.0: 16, 37.5: 12, 87.5: 5, 25.0: 4, 100.0: 2, 12.5: 1})
Valence 62.125
Counter({75.0: 26, 62.5: 25, 50.0: 23, 87.5: 10, 37.5: 10, 25.0: 4, 100.0: 2})
Arousal Valence
[59.0, 62.125, 0.5297690642690643, 0.6159860972360973]
s15
Array of average Happy Sad accurecies for user s15
Counter({50.0: 45, 75.0: 31, 25.0: 15, 100.0: 5, 0.0: 4})
Happy Sad
[54.5, 0.5336190476190477]
Array of average All Emotions accurecies for user s15
Counter({25.0: 38, 37.5: 25, 12.5: 17, 50.0: 10, 0.0: 7, 62.5: 2, 87.5: 1})
Combined Emotions
[28.125, 0.27225198412698415]
Arousal 49.0
Counter({50.0: 34, 62.5: 22, 37.5: 21, 25.0: 14, 75.0: 8, 87.5: 1})
Valence 56.375
Counter({50.0: 28, 62.5: 22, 75.0: 20, 37.5: 17, 87.5: 7, 25.0: 3, 12.5: 3})
Arousal Valence
[49.0, 56.375, 0.4634101731601732, 0.5602649572649573]
s16
Array of average Happy Sad accurecies for user s16
Counter({80.0: 29, 40.0: 24, 60.0: 21, 100.0: 19, 20.0: 6, 0.0: 1})
Happy Sad
[65.6, 0.6373809523809524]
Arr

Array of average Happy Sad accurecies for user s28
Counter({83.33333333333334: 30, 66.66666666666666: 30, 50.0: 27, 100.0: 8, 33.33333333333333: 4, 16.666666666666664: 1})
Happy Sad
[68.0, 0.6782721260221258]
Array of average All Emotions accurecies for user s28
Counter({25.0: 33, 37.5: 24, 50.0: 18, 12.5: 17, 0.0: 4, 62.5: 3, 75.0: 1})
Combined Emotions
[31.0, 0.29733450577200576]
Arousal 53.0
Counter({50.0: 37, 62.5: 24, 37.5: 18, 75.0: 13, 25.0: 4, 87.5: 2, 12.5: 2})
Valence 64.75
Counter({62.5: 24, 50.0: 23, 75.0: 23, 87.5: 16, 37.5: 6, 100.0: 4, 12.5: 2, 25.0: 2})
Arousal Valence
[53.0, 64.75, 0.5135021645021645, 0.6535777555777557]
s29
Array of average Happy Sad accurecies for user s29
Counter({83.33333333333334: 35, 66.66666666666666: 33, 50.0: 15, 100.0: 13, 33.33333333333333: 3, 16.666666666666664: 1})
Happy Sad
[72.83333333333334, 0.725308561808562]
Array of average All Emotions accurecies for user s29
Counter({37.5: 25, 62.5: 23, 50.0: 22, 25.0: 16, 12.5: 8, 75.0: 4, 0.0: 1,